In [1]:
import pandas as pd 
import geopy.distance
import os
import json
import pickle
import uuid
import warnings
from concurrent.futures import ThreadPoolExecutor
from datetime import datetime
import boto3
import numpy as np
import pandas as pd
from dateutil import relativedelta
from scipy.stats import randint as sp_randint
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.externals import joblib
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
import holidays
from pandas.tseries.holiday import USFederalHolidayCalendar
import xgboost

In [2]:
cd ..


/Users/nathantoubiana/Documents/Codes/Kaggle


In [3]:
cd all

/Users/nathantoubiana/Documents/Codes/Kaggle/all


In [ ]:
df=pd.read_csv('train.csv')

In [ ]:
def dist(lat1,long1,lat2,long2):
    try:
        coords_1 = (long1,lat1)
        coords_2 = (long2,lat2)

        return(geopy.distance.vincenty(coords_1, coords_2).km)
    except:
        return 'bug'

In [ ]:
def lat_dist(lat1,long1,lat2,long2):
    return dist(lat1,long1,lat2,long1)

In [ ]:
def long_dist(lat1,long1,lat2,long2):
    return dist(lat1,long1,lat1,long2)

In [ ]:
df['distance']=df.apply(lambda x: dist(x.pickup_latitude,x.pickup_longitude,x.dropoff_latitude,x.dropoff_longitude),axis=1)

In [ ]:
df=df[df.distance!='bug']

In [ ]:
df['latdistance']=df.apply(lambda x: lat_dist(x.pickup_latitude,x.pickup_longitude,x.dropoff_latitude,x.dropoff_longitude),axis=1)

In [ ]:
df['longdistance']=df.apply(lambda x: long_dist(x.pickup_latitude,x.pickup_longitude,x.dropoff_latitude,x.dropoff_longitude),axis=1)

In [ ]:
df=df[df.distance!='bug']

In [49]:
df.head()

,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,distance,mandistance
0,2009-06-15 17:26:21.0000001,4.5,2009-06-15 17:26:21 UTC,-73.844311,40.721319,-73.841610,40.712278,1,0.412061,0.582358
1,2010-01-05 16:52:16.0000002,16.9,2010-01-05 16:52:16 UTC,-74.016048,40.711303,-73.979268,40.782004,1,4.64623,6.278943
2,2011-08-18 00:35:00.00000049,5.7,2011-08-18 00:35:00 UTC,-73.982738,40.761270,-73.991242,40.750562,2,1.0048,1.279045
3,2012-04-21 04:30:42.0000001,7.7,2012-04-21 04:30:42 UTC,-73.987130,40.733143,-73.991567,40.758092,1,0.914154,1.263716
4,2010-03-09 07:51:00.000000135,5.3,2010-03-09 07:51:00 UTC,-73.968095,40.768008,-73.956655,40.783762,1,1.36616,1.762630


In [200]:
y = df['fare_amount'].dropna(axis=0, how='any').as_matrix()

In [202]:
X.shape

(9999545, 6)

In [201]:
X = (df[[ 'pickup_longitude',
       'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude',
       'passenger_count','distance']]
                 .dropna(axis=0, how='any')
                 .as_matrix())

In [203]:
clf = RandomForestRegressor(max_depth=15, max_features=5, n_estimators=100,n_jobs=-1)

In [57]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=400)
clf.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=15,
           max_features=5, max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=100, n_jobs=-1, oob_score=False, random_state=None,
           verbose=0, warm_start=False)

In [204]:
clf.fit(X, y)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=15,
           max_features=5, max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=100, n_jobs=-1, oob_score=False, random_state=None,
           verbose=0, warm_start=False)

In [205]:
df_test=pd.read_csv('test.csv')

In [206]:
def dist(lat1,long1,lat2,long2):
    try:
        coords_1 = (long1,lat1)
        coords_2 = (long2,lat2)

        return(geopy.distance.vincenty(coords_1, coords_2).km)
    except:
        return 'bug'

In [207]:
df_test['distance']=df_test.apply(lambda x: dist(x.pickup_latitude,x.pickup_longitude,x.dropoff_latitude,x.dropoff_longitude),axis=1)

In [208]:
X = (df_test[['pickup_datetime', 'pickup_longitude',
       'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude',
       'passenger_count','distance']]
                 ._get_numeric_data()
                 .as_matrix())

In [209]:
predicted = clf.predict(X)



In [210]:
df_test['preds']=predicted

In [211]:
df_test_keys=df_test[['key','preds']]

In [212]:
df_final=pd.read_csv('sample_submission.csv')

In [213]:
df_final.fare_amount=df_final.merge(df_test_keys,on='key')['preds']

In [214]:
df_final[['key','fare_amount']].to_csv('results_nathan.csv' ,  index=False)

In [215]:
s = pickle.dumps(clf)
joblib.dump(clf, 'baseline_model.pkl')

['baseline_model.pkl']

MORE FEATURES

In [13]:
df['pickup_datetime'] = df['pickup_datetime'].str.slice(0, 16)
df['pickup_datetime'] = pd.to_datetime(df['pickup_datetime'], utc=True, format='%Y-%m-%d %H:%M')

In [14]:
def transform(data):
    # Extract date attributes and then drop the pickup_datetime column
    data['hour'] = data['pickup_datetime'].dt.hour
    data['day'] = data['pickup_datetime'].dt.day
    data['month'] = data['pickup_datetime'].dt.month
    data['year'] = data['pickup_datetime'].dt.year
    data['weekday']=data['pickup_datetime'].dt.weekday
    return data

In [15]:
df1=transform(df)

In [38]:
df1.head()

,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,distance,mandistance,hour,day,month,year,weekday
0,2009-06-15 17:26:21.0000001,4.5,2009-06-15 17:26:00,-73.844311,40.721319,-73.841610,40.712278,1,0.412061,0.582358,17,15,6,2009,0
1,2010-01-05 16:52:16.0000002,16.9,2010-01-05 16:52:00,-74.016048,40.711303,-73.979268,40.782004,1,4.64623,6.278943,16,5,1,2010,1
2,2011-08-18 00:35:00.00000049,5.7,2011-08-18 00:35:00,-73.982738,40.761270,-73.991242,40.750562,2,1.0048,1.279045,0,18,8,2011,3
3,2012-04-21 04:30:42.0000001,7.7,2012-04-21 04:30:00,-73.987130,40.733143,-73.991567,40.758092,1,0.914154,1.263716,4,21,4,2012,5
4,2010-03-09 07:51:00.000000135,5.3,2010-03-09 07:51:00,-73.968095,40.768008,-73.956655,40.783762,1,1.36616,1.762630,7,9,3,2010,1


In [21]:
#df1.to_csv('train_with_features.csv')

In [44]:
def hol(x):
    if x in holidays:
        return 1
    else:
        return 0

In [49]:
df1.shape

(55421320, 15)

In [16]:
y = df1['fare_amount'].dropna(axis=0, how='any').as_matrix()

In [17]:
X = (df1[[ 'pickup_longitude',
       'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude',
       'passenger_count','distance','year','month','day','hour','weekday','mandistance']]
                 .dropna(axis=0, how='any')
                 .as_matrix())

In [56]:
X.shape

(55421320, 12)

In [18]:

#TO DO: 

#add holidays
#retrain with larger max depth
#add features about location zones (kmeans?)
#remove absurd outliers (water, etc)
#try radius NN
#try ANNOY?
#try ensembles of above models? add meta learning?


In [ ]:
clf = xgboost.XGBRegressor(max_depth=20, max_features=5, n_estimators=100,n_jobs=-1)
#make max depth larger : 20

In [ ]:
clf.fit(X, y)

In [64]:
df_test=pd.read_csv('test.csv')

In [65]:
df_test['distance']=df_test.apply(lambda x: dist(x.pickup_latitude,x.pickup_longitude,x.dropoff_latitude,x.dropoff_longitude),axis=1)

In [66]:
df_test['mandistance']=df_test.apply(lambda x: man_dist(x.pickup_latitude,x.pickup_longitude,x.dropoff_latitude,x.dropoff_longitude),axis=1)

In [67]:
df_test['pickup_datetime'] = df_test['pickup_datetime'].str.slice(0, 16)
df_test['pickup_datetime'] = pd.to_datetime(df_test['pickup_datetime'], utc=True, format='%Y-%m-%d %H:%M')

In [68]:
df_test=transform(df_test)

In [69]:
X = (df_test[['pickup_longitude',
       'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude',
       'passenger_count','distance','year','month','day','hour','weekday','mandistance']]
                 ._get_numeric_data()
                 .as_matrix())

In [70]:
X.shape

(9914, 12)

In [71]:
predicted = clf.predict(X)



In [72]:
df_test['preds']=predicted
df_test_keys=df_test[['key','preds']]
df_final=pd.read_csv('sample_submission.csv')
df_final.fare_amount=df_final.merge(df_test_keys,on='key')['preds']
df_final[['key','fare_amount']].to_csv('results_nathan.csv' ,  index=False)

In [73]:
s = pickle.dumps(clf)
joblib.dump(clf, 'model_3_man.pkl')

['model_3_man.pkl']

In [3]:
inputs=[]
outputs=[]

In [9]:
for i in [100]:
    for k in [10,50]:
        for p in [5,50]:
                for n in [100,1000]:
                    outputs.append(measure_time(j=2,i=i,n=n,k=k,p=p))
                    inputs.append(np.array([n,p,i,2,k,-1]))

In [5]:
def measure_time(n=10,p=10,i=5,j=2,k=100,l=-1):
    start_time = time.time()
    X=np.random.rand(n,p)
    y=np.random.rand(n,)
    clf = RandomForestRegressor(max_depth=i, max_features=j, n_estimators=k,n_jobs=l)
    clf.fit(X,y)
    elapsed_time = time.time() - start_time
    return elapsed_time

In [7]:
import time 

In [11]:
y=np.array(outputs)
X=np.array(inputs)

In [47]:
clf = xgboost.XGBRegressor(max_depth=20, max_features=5,eval_metric="rmse", n_estimators=10000,verbose=True,
                           n_jobs=-1,eval_set=(X,y))

In [48]:
clf.fit(X,y)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, eval_metric='rmse',
       eval_set=(array([[ 100,    5,  100,    2,   10,   -1],
       [1000,    5,  100,    2,   10,   -1],
       [ 100,   50,  100,    2,   10,   -1],
       [1000,   50,  100,    2,   10,   -1],
       [ 100,    5,  100,    2,   50,   -1],
       [1000,    5,  100,    2,   50,   -1],
       [ 100,   50,  100,    2,   50,   -1],
       [1000,   50,  100,    2,   50,   -1]]), array([ 0.11024,  0.11108,  0.11011,  0.10954,  0.13233,  0.13724,
        0.13642,  0.13366])),
       gamma=0, learning_rate=0.1, max_delta_step=10, max_depth=20,
       max_features=5, min_child_weight=1, missing=None,
       n_estimators=10000, n_jobs=-1, nthread=None, objective='reg:linear',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=1, verbose=True)

In [42]:
from numpy import loadtxt
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
# load data
dataset = loadtxt('pima-indians-diabetes.csv', delimiter=",")
# split data into X and y
X = dataset[:,0:8]
Y = dataset[:,8]
# split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=7)
# fit model no training data
model = XGBClassifier()
eval_set = [(X_test, y_test)]
model.fit(X_train, y_train, eval_metric="error", eval_set=eval_set, verbose=True)
# make predictions for test data
y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]
# evaluate predictions
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

FileNotFoundError: [Errno 2] No such file or directory: 'pima-indians-diabetes.csv'